In [1]:
import requests
import datetime as dt
import time
import json
import pytz
import threading
from sqlalchemy.dialects.postgresql import insert
import pandas as pd
pd.set_option('display.max_columns', None)

from database import get_db
from models import *

In [2]:
db_gen = get_db()
db = next(db_gen)

In [11]:
latest_value, latest_date = db.query(CryptocurrencyHistory.Value, CryptocurrencyHistory.Date_Local) \
    .filter(CryptocurrencyHistory.Crypto_Id == 'teter') \
        .order_by(CryptocurrencyHistory.Datetime_Local.desc()) \
            .first()

In [12]:
latest_date_1 = latest_date - dt.timedelta(days= 1)
latest_date_7 = latest_date - dt.timedelta(days= 7)
latest_date_30 = latest_date - dt.timedelta(days= 30)

In [13]:
latest_value_1 = db.query(CryptocurrencyHistory.Value) \
    .filter(CryptocurrencyHistory.Crypto_Id == 'teter', CryptocurrencyHistory.Date_Local == latest_date_1) \
        .order_by(CryptocurrencyHistory.Datetime_Local.desc()) \
            .first()[0]

latest_value_7 = db.query(CryptocurrencyHistory.Value) \
    .filter(CryptocurrencyHistory.Crypto_Id == 'teter', CryptocurrencyHistory.Date_Local == latest_date_7) \
        .order_by(CryptocurrencyHistory.Datetime_Local.desc()) \
            .first()[0]

latest_value_30 = db.query(CryptocurrencyHistory.Value) \
    .filter(CryptocurrencyHistory.Crypto_Id == 'teter', CryptocurrencyHistory.Date_Local == latest_date_30) \
        .order_by(CryptocurrencyHistory.Datetime_Local.desc()) \
            .first()[0]

In [14]:
daily_rate_of_change = (latest_value - latest_value_1) / latest_value_1
weekly_rate_of_change = (latest_value - latest_value_7) / latest_value_7
monthly_rate_of_change = (latest_value - latest_value_30) / latest_value_30

In [17]:
records = {
    'Crypto_Id': 'teter',
    'Date_Local': latest_date,
    'Latest_Value': latest_value,
    'Daily_Rate_Of_Change': daily_rate_of_change,
    'Weekly_Rate_Of_Change': weekly_rate_of_change,
    'Monthly_Rate_Of_Change': monthly_rate_of_change
}

In [21]:
stmt = insert(CryptocurrencyState).values(records)
stmt = stmt.on_conflict_do_update(
index_elements =['Crypto_Id', 'Date_Local'],
set_={
    'Latest_Value': stmt.excluded.Latest_Value,
    'Daily_Rate_Of_Change': stmt.excluded.Daily_Rate_Of_Change,
    'Weekly_Rate_Of_Change': stmt.excluded.Weekly_Rate_Of_Change,
    'Monthly_Rate_Of_Change': stmt.excluded.Monthly_Rate_Of_Change
    }
)

In [22]:
try:
    db.execute(stmt)
    db.commit()
except Exception as e:
    # Rollback the transaction in case of error
    db.rollback()
    raise e
finally:
    db.close()

In [3]:
from_time = dt.datetime(2024, 6, 1, tzinfo= pytz.timezone('Asia/Tehran'))
from_timestamp = str(int(from_time.timestamp()))
to_time = dt.datetime.now()
to_timestamp = str(int(to_time.timestamp()))

crypto_metadata = {
    'teter': 'usdt-irt',
    'bitcoin': 'btc-irt'
}

raw_data = get_cryptocurrency_historic_data('usdt-irt', from_timestamp, to_timestamp)
# df = transform_cryptocurrency_historic_data(raw_data, 'teter')

https://api.exir.io/v2/chart?symbol=usdt-irt&resolution=15&from=1717187640&to=1717685821


In [4]:
raw_data

[{'time': '2024-05-31T20:45:00.000Z',
  'close': 59130,
  'high': 59130,
  'low': 59129,
  'open': 59129,
  'symbol': 'usdt-irt',
  'volume': 4110.38},
 {'time': '2024-05-31T21:00:00.000Z',
  'close': 59130,
  'high': 59130,
  'low': 59130,
  'open': 59130,
  'symbol': 'usdt-irt',
  'volume': 422.73},
 {'time': '2024-05-31T21:15:00.000Z',
  'close': 58859,
  'high': 59130,
  'low': 58859,
  'open': 59130,
  'symbol': 'usdt-irt',
  'volume': 42.76},
 {'time': '2024-05-31T21:30:00.000Z',
  'close': 58500,
  'high': 59129,
  'low': 58500,
  'open': 59129,
  'symbol': 'usdt-irt',
  'volume': 5538.08},
 {'time': '2024-05-31T21:45:00.000Z',
  'close': 59099,
  'high': 59099,
  'low': 58436,
  'open': 58750,
  'symbol': 'usdt-irt',
  'volume': 3716.4500000000003},
 {'time': '2024-05-31T22:00:00.000Z',
  'close': 59099,
  'high': 59099,
  'low': 58768,
  'open': 58934,
  'symbol': 'usdt-irt',
  'volume': 2804.8500000000004},
 {'time': '2024-05-31T22:45:00.000Z',
  'close': 58768,
  'high': 587

In [20]:
from_timestamp

'1740774840'

In [21]:
to_timestamp

'1716567573'

In [19]:
raw_data

[]

In [12]:
df = transform_cryptocurrency_historic_data(raw_data, 'teter')

In [15]:
pd.DataFrame(raw_data).columns

Index(['time', 'close', 'high', 'low', 'open', 'symbol', 'volume'], dtype='object')

In [4]:
df = pd.DataFrame(raw_data)

In [6]:
import pytz

In [10]:
df.iloc[0]['time']

'2024-02-29T20:30:00.000Z'

In [14]:
df['time2'] = pd.to_datetime(df['time'])
df['time3'] = df['time2'].dt.tz_convert(pytz.timezone('Asia/Tehran'))

In [15]:
df

,time,close,high,low,open,symbol,volume,time2,time3
0,2024-02-29T20:30:00.000Z,58779,58779,58750,58750,usdt-irt,958.33,2024-02-29 20:30:00+00:00,2024-03-01 00:00:00+03:30
1,2024-02-29T21:00:00.000Z,58779,58779,58690,58690,usdt-irt,276.10,2024-02-29 21:00:00+00:00,2024-03-01 00:30:00+03:30
2,2024-02-29T21:15:00.000Z,58779,58779,58629,58734,usdt-irt,633.17,2024-02-29 21:15:00+00:00,2024-03-01 00:45:00+03:30
3,2024-02-29T21:30:00.000Z,58700,58700,58700,58700,usdt-irt,6474.93,2024-02-29 21:30:00+00:00,2024-03-01 01:00:00+03:30
4,2024-02-29T21:45:00.000Z,58550,58700,58400,58700,usdt-irt,3515.83,2024-02-29 21:45:00+00:00,2024-03-01 01:15:00+03:30
...,...,...,...,...,...,...,...,...,...
6839,2024-05-24T14:45:00.000Z,58164,58164,57832,57886,usdt-irt,233.83,2024-05-24 14:45:00+00:00,2024-05-24 18:15:00+03:30
6840,2024-05-24T15:00:00.000Z,57703,58164,57703,58164,usdt-irt,1179.39,2024-05-24 15:00:00+00:00,2024-05-24 18:30:00+03:30
6841,2024-05-24T15:15:00.000Z,57672,57703,57643,57703,usdt-irt,6136.00,2024-05-24 15:15:00+00:00,2024-05-24 18:45:00+03:30
6842,2024-05-24T15:30:00.000Z,57672,57672,57644,57644,usdt-irt,2307.03,2024-05-24 15:30:00+00:00,2024-05-24 19:00:00+03:30
